In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import SAGEConv, GAE
from torch.optim.lr_scheduler import StepLR
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
data = pd.read_csv("./data/train.csv")

In [ ]:
users = data[['User-ID', 'Age']].drop_duplicates().reset_index(drop=True)
books = data[['Book-ID', 'Year-Of-Publication']].drop_duplicates().reset_index(drop=True)

user_id_map = {user_id: idx for idx, user_id in enumerate(users['User-ID'].unique())}
book_id_map = {book_id: idx for idx, book_id in enumerate(books['Book-ID'].unique())}

data['Age'] = (users['Age'] - users['Age'].min()) / (users['Age'].max() - users['Age'].min())
data['Year-Of-Publication'] = (books['Year-Of-Publication'] - books['Year-Of-Publication'].min()) / (books['Year-Of-Publication'].max() - books['Year-Of-Publication'].min())

data['User-ID'] = data['User-ID'].map(user_id_map)
data['Book-ID'] = data['Book-ID'].map(book_id_map)

data_processed = data.fillna(0)

In [ ]:
node_features = data_processed[["Age","Year-Of-Publication"]].to_numpy()
edge_index = torch.tensor(data_processed[['User-ID', 'Book-ID']].values, dtype=torch.long).t().contiguous()

#################################### 더 적합한 것으로 추후 변경
edge_attr = torch.tensor(data_processed['Book-Rating'].values, dtype=torch.float).unsqueeze(-1)

target = torch.tensor(data_processed['Book-Rating'].values, dtype=torch.float)

In [ ]:
print(edge_index)
print(edge_index.shape)
print(f"Number of rows in data_processed: {len(data_processed)}")

# Make data

graph_data = Data(x=torch.tensor(node_features), edge_index=edge_index, edge_attr=edge_attr)
graph_data.n_id = torch.arange(graph_data.num_nodes)

In [ ]:
graph_data.x

In [ ]:
graph_data.edge_index

In [ ]:
graph_data.edge_attr

In [ ]:
graph_data.n_id

# Data Split

In [ ]:
# 데이터를 훈련 및 테스트 데이터로 분할합니다. (예를 들어 80:20 비율)
total_data_len = len(graph_data)
train_data_len = int(total_data_len * 0.8)
test_data_len = total_data_len - train_data_len

train_data, test_data = random_split(graph_data, [train_data_len, test_data_len])

train_loader = DataLoader(train_data, batch_size=512, shuffle=True, num_workers=6)
test_loader = DataLoader(test_data, batch_size=512, shuffle=False, num_workers=6)

In [ ]:
### Neighbor Loader
# gSAGE_loader = NeighborLoader(
#     graph_data,
#     num_neighbors=[10,10],
#     batch_size=512)

# GraphSAGE

In [ ]:
in_channels = 32
hidden_channels = 64
out_channels = 1
####################################
# 추후 베이지안 최적화(Bayesian optimization)
####################################
# conv 레이어 개수
# 은닉층의 노드 수 (hidden_channels)
# 드롭아웃 비율 (dropout rate)
# 학습률 (learning rate)
# 가중치 감소 (weight decay)
# 배치 크기 (batch size)
# 최적화 알고리즘 (optimizer)

In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        return x

In [ ]:
model = GraphSAGE(in_channels=in_channels, hidden_channels=hidden_channels, out_channels=out_channels)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005,weight_decay=1e-5)
loader = DataLoader(dataset, batch_size=1, shuffle=True)
loss_function = torch.nn.MSELoss()

In [ ]:
def train(loader):
    model.train()
    total_loss = 0
    for data in tqdm(loader):
        data = data.to(device)
        optimizer.zero_grad()
        z = model(data.x.float(), data.edge_index)
        
        book_ratings = data.y.to(device)
        
        loss = loss_function(z, book_ratings)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader), model

In [ ]:
num_epoch = 100

for epoch in range(0,num_epoch):
    loss,model = train(loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')